<a href="https://colab.research.google.com/github/Ronit-Wanare/Customer-Conversion-Classification-Model/blob/main/Audiobook_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from google.colab import files
import tensorflow as tf

Tranfering data and preprocessing

In [ ]:
raw_csv_data = np.loadtxt('/content/Audiobooks_data.csv',delimiter=',')
unscaled_input_all = raw_csv_data[:,1:-1]
target_all = raw_csv_data[:,-1]

In [ ]:
unscaled_input_all

array([[1620.  , 1620.  ,   19.73, ..., 1603.8 ,    5.  ,   92.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,    0.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,  388.  ],
       ...,
       [2160.  , 2160.  ,    6.14, ...,    0.  ,    0.  ,    0.  ],
       [1620.  , 1620.  ,    5.33, ...,  615.6 ,    0.  ,   90.  ],
       [1674.  , 3348.  ,    5.33, ...,    0.  ,    0.  ,    0.  ]])

Balancing

In [ ]:
num_one_targets = int(np.sum(target_all))
zero_target_counter = 0
indices_to_remove = []
# for iteration
for i in range(target_all.shape[0]):
  if target_all[i] == 0:
    zero_target_counter += 1
    if zero_target_counter > num_one_targets:
      indices_to_remove.append(i)
# removing the indices
unscaled_inputs_equal_priors = np.delete(unscaled_input_all,indices_to_remove,axis = 0)
targets_equal_priors = np.delete(target_all,indices_to_remove,axis = 0)

Standardising the input

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

shuffle the data

In [ ]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

Spliting the data into train,validation and test sets

In [ ]:
sample_count = shuffled_inputs.shape[0]
#defing sizes (80:10:10)
train_sample_count = int(0.8 * sample_count)
validation_sample_count = int(0.1 * sample_count)
test_sample_count = sample_count - train_sample_count - validation_sample_count

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count + validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count + validation_sample_count]

test_inputs = shuffled_inputs[train_sample_count + validation_sample_count:]
test_targets = shuffled_targets[train_sample_count + validation_sample_count:]
#checkiing the split
print(np.sum(train_targets),train_sample_count, np.sum(train_targets)/train_sample_count)
print(np.sum(validation_targets),validation_sample_count, np.sum(validation_targets)/validation_sample_count)
print(np.sum(test_targets),test_sample_count, np.sum(test_targets)/test_sample_count)

1802.0 3579 0.5034925956971221
217.0 447 0.4854586129753915
218.0 448 0.48660714285714285


Save the three datasets in .npz format


In [ ]:
np.savez('Audiobooks_data_train',inputs = train_inputs , outputs = train_targets)
np.savez('Audiobooks_data_validation',inputs = validation_inputs , outputs = validation_targets)
np.savez('Audiobooks_data_test',inputs = test_inputs , outputs = test_targets)

CREATING ML ALGO

Data

In [ ]:
# loading train dataset
npz = np.load('/content/Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['outputs'].astype(int)
# loading validation dataset
npz = np.load('/content/Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['outputs'].astype(int)

# loading tst dataset
npz = np.load('/content/Audiobooks_data_test.npz')

test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['outputs'].astype(int)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launch

Model

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
            
            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
            tf.keras.layers.Dense(output_size, activation='softmax')
])

Optimizer and Loss function

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

FIt the model

In [ ]:
batch_size = 100
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data =(validation_inputs,validation_targets),
          verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.5765 - accuracy: 0.7002 - val_loss: 0.4949 - val_accuracy: 0.7651 - 1s/epoch - 28ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4598 - accuracy: 0.7700 - val_loss: 0.4208 - val_accuracy: 0.7718 - 89ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4140 - accuracy: 0.7754 - val_loss: 0.3890 - val_accuracy: 0.7987 - 150ms/epoch - 4ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3916 - accuracy: 0.7879 - val_loss: 0.3782 - val_accuracy: 0.7875 - 103ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3757 - accuracy: 0.7969 - val_loss: 0.3605 - val_accuracy: 0.8121 - 99ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3664 - accuracy: 0.7999 - val_loss: 0.3524 - val_accuracy: 0.8210 - 91ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3583 - accuracy: 0.8064 - val_loss: 0.3462 - val_accuracy: 0.8255 - 97ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3546 - accuracy: 0.8036 - val_loss: 0.3491 - val_accuracy: 0.8188 - 104ms/epoch - 3ms/step
Epoch 9/100
36

Test the model

In [ ]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)
print('Test loss: {0:.2f}.Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


14/14 [==============================] - 0s 2ms/step - loss: 0.3056 - accuracy: 0.8460
Test loss: 0.31.Test accuracy: 84.60%
